DNN testing playground! 

i wanna put all the final shit for the DNN into a .py file but for now while I fuck with stuff, ill do it here. that is if this bad boi can even handle the big ass .h5py files I made, if not either make smaller ones or just use alpha.

In [2]:
# boilerplate, ensure you have these packages
import numpy as np
import tensorflow as tf
from keras import callbacks
from keras.callbacks import LearningRateScheduler as LRS
from keras.models import Model
from keras.layers import *
from keras import optimizers as kopt
import keras.backend as K
import datetime
import h5py
import hyperopt as hopt

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3675537013496737362
]


In [6]:
65./90.

0.7222222222222222

In [ ]:
# data rescaling functions, want everything to be [0,1]
# rescaling thicknesses, angle of incid, psi, delta
# this info should be pulled from the corresponding .txt file!!!
# ^although im not quite sure how I wanna do tht rn so just list it in the .txt file
def resc_l(thickness):
    'rescales the thicknesses to [0,1], takes in arrays'
    resc = thickness*1e7 
    return resc
def resc_ang(aoi):
    'rescales the aoi to [0,1], takes in arrays in DEGREES'
    resc = aoi/90.
    return resc
def resc_ellips(ellips):
    '''
    rescales ellipsometric coeff psi and delta to [0,1], takes in array in RADS
    able to take in negative numbers
    '''